In [84]:
import imagej
import os
import scyjava as sj
from scyjava import jimport
import random
import sys
sj.config.add_options('-Xmx6g')

os.environ["JAVA_HOME"] = "/Users/nelsschimek/mambaforge/envs/i2k-2022-pyimagej/"

In [2]:
# initialize ImageJ
ij = imagej.init('/Applications/Fiji.app/')
print(f"ImageJ version: {ij.getVersion()}")

ImageJ version: 2.9.0/1.53t


In [3]:
Runtime = jimport('java.lang.Runtime')

In [4]:
print(Runtime.getRuntime().maxMemory() // (2**20), " MB available to java")

5461  MB available to java


In [63]:
image_path = "/Users/nelsschimek/Documents/nancelab/diff_classifier/notebooks/development/MPT_Data/P10F_NT_10DIV_40nm_slice_2_midbrain_vid_2_0_0.tif"
#image_path = "/Users/nelsschimek/Documents/nancelab/Data/QD_BEV_tifs/60X_QD-BEV_Video1.tif"
image = ij.io().open(image_path)

In [64]:
image

<java object 'net.imagej.DefaultDataset'>

In [65]:
def dump_info(image):
    """A handy function to print details of an image object."""
    name = image.name if hasattr(image, 'name') else None # xarray
    if name is None and hasattr(image, 'getName'): name = image.getName() # Dataset
    if name is None and hasattr(image, 'getTitle'): name = image.getTitle() # ImagePlus
    print(f" name: {name or 'N/A'}")
    print(f" type: {type(image)}")
    print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"shape: {image.shape}")
    print(f" dims: {image.dims if hasattr(image, 'dims') else 'N/A'}")

if not 'image' in locals(): print("I cannot find any variable called embryo. Did you write `embryo = ...` above?")
elif str(type(image)) != "<java class 'net.imagej.DefaultDataset'>": print("The embryo variable does not contain an ImageJ2 Dataset. Did you use the `ij.io().open` function?")
else: dump_info(image)

 name: P10F_NT_10DIV_40nm_slice_2_midbrain_vid_2_0_0.tif
 type: <java class 'net.imagej.DefaultDataset'>
dtype: <java class 'net.imglib2.type.numeric.integer.UnsignedShortType'>
shape: (512, 512, 651)
 dims: ('X', 'Y', 'Unknown')


In [8]:
image.__dict__

{}

In [75]:
Model = sj.jimport('fiji.plugin.trackmate.Model')
Settings = sj.jimport('fiji.plugin.trackmate.Settings')
TrackMate = sj.jimport('fiji.plugin.trackmate.TrackMate')
FeatureFilter = sj.jimport('fiji.plugin.trackmate.features.FeatureFilter')
LAPUtils =  sj.jimport('fiji.plugin.trackmate.tracking.jaqaman.LAPUtils')
SelectionModel = sj.jimport('fiji.plugin.trackmate.SelectionModel')
Logger = sj.jimport('fiji.plugin.trackmate.Logger')
DisplaySettingsIO = sj.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
HyperStackDisplayer = sj.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')

DogDetectorFactory = sj.jimport('fiji.plugin.trackmate.detection.DogDetectorFactory')
SparseLAPTrackerFactory = sj.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')

In [99]:
model = Model()

In [100]:
model.setLogger(Logger.IJ_LOGGER)

In [15]:

fake_im = ij.io().open('https://fiji.sc/samples/FakeTracks.tif')


In [101]:
imp = ij.py.to_imageplus(image)

In [102]:
original_dims = imp.getDimensions()
for val in original_dims:
    print(val)

512
512
651
1
1


In [103]:
dims = imp.getDimensions() # default order: XYCZT
print(imp.dims)
dump_info(imp)

('X', 'Y', 'C')
 name: P10F_NT_10DIV_40nm_slice_2_midbrain_vid_2_0_0.tif
 type: <java class 'ij.ImagePlus'>
dtype: N/A
shape: (512, 512, 651)
 dims: ('X', 'Y', 'C')


In [104]:
# Swap Z and T dimensions if T=1

if (dims[4] == 1):
	print('need to change order')
	imp.setDimensions(dims[4], dims[3], dims[2])
print(imp.dims)

# Setup settings for TrackMate

need to change order
('X', 'Y', 'T')


In [105]:
for val in imp.getDimensions():
    print(val)

512
512
1
1
651


In [106]:
for val in original_dims:
    print(val)

512
512
651
1
1


In [107]:
dump_info(imp)

 name: P10F_NT_10DIV_40nm_slice_2_midbrain_vid_2_0_0.tif
 type: <java class 'ij.ImagePlus'>
dtype: N/A
shape: (512, 512, 651)
 dims: ('X', 'Y', 'T')


In [108]:
for i in range(5):
    print(dims[i])

512
512
651
1
1


In [109]:
imp.dims

('X', 'Y', 'T')

In [110]:
settings = Settings(imp)

In [111]:
# Configure detector
settings.detectorFactory = DogDetectorFactory()
settings.detectorSettings = {
    'DO_SUBPIXEL_LOCALIZATION' : False,
    'RADIUS' : 6.0,
    'TARGET_CHANNEL': ij.py.to_java(1),
    'DO_MEDIAN_FILTERING': True,
    'THRESHOLD': 0.0
}  

In [112]:
# TMUtils = sj.jimport('fiji.plugin.trackmate.util.TMUtils') #.otsuThreshold(featureValues)

# trackFeatureValues = model.getFeatureModel.getTrackFeatureValues()
# featureValues = trackFeatureValues.get('NUMBER_SPOTS')
# optimalNumberSpots = TMUtils.otsuThreshold(featureValues)

In [113]:
# filter1 = FeatureFilter('QUALITY', 3.26, True) #Needs to be (str, str, bool)
# settings.addSpotFilter(filter1)

In [114]:
# Configure tracker
settings.trackerFactory = SparseLAPTrackerFactory()
settings.trackerSettings = LAPUtils.getDefaultSegmentSettingsMap()
settings.trackerSettings['LINKING_MAX_DISTANCE'] = 15.0
settings.trackerSettings['GAP_CLOSING_MAX_DISTANCE'] = 20.0
settings.trackerSettings['MAX_FRAME_GAP'] = ij.py.to_java(6)

In [115]:
trackmate = TrackMate(model, settings)
#trackmate.execDetection()

In [116]:
ok = trackmate.checkInput()
if not ok:
    sys.exit(str(trackmate.getErrorMessage()))

ok = trackmate.process()
if not ok:
    sys.exit(str(trackmate.getErrorMessage()))

Starting detection process using 8 threads.

Detection processes 8 frames simultaneously and allocates 1 thread per frame.



[java.lang.Enum.toString] Starting detection process using 8 threads.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Detection processes 8 frames simultaneously and allocates 1 thread per frame.
[java.lang.Enum.toString] 


Found 793453 spots.

Starting initial filtering process.



[java.lang.Enum.toString] Found 793453 spots.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting initial filtering process.
[java.lang.Enum.toString] 


Computing spot features over 8 frames simultaneously and allocating 1 thread per frame.

Computation done in 218 ms.

Starting spot filtering process.



[java.lang.Enum.toString] Computing spot features over 8 frames simultaneously and allocating 1 thread per frame.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computation done in 218 ms.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting spot filtering process.
[java.lang.Enum.toString] 


Starting tracking process.



[java.lang.Enum.toString] Starting tracking process.
[java.lang.Enum.toString] 


In [ ]:
model.getLogger().log('Found ' + str(model.getTrackModel().nTracks(True)) + ' tracks.')


Found 87 tracks.


[java.lang.Enum.toString] Found 87 tracks.[java.lang.Enum.toString] 


In [80]:
# A selection.
sm = SelectionModel( model )

# Read the default display settings.
ds = DisplaySettingsIO.readUserDefault()

# The viewer.
displayer =  HyperStackDisplayer( model, sm, imp, ds ) 
#displayer.render()

# The feature model, that stores edge and track features.
fm = model.getFeatureModel()

In [81]:
# Iterate over all the tracks that are visible.
for id in model.getTrackModel().trackIDs(True):

    # Fetch the track feature from the feature model.
    v = fm.getTrackFeature(id, 'TRACK_MEAN_SPEED')
    model.getLogger().log('')
    model.getLogger().log('Track ' + str(id) + ': mean velocity = ' + str(v) + ' ' + model.getSpaceUnits() + '/' + model.getTimeUnits())

	# Get all the spots of the current track.
    track = model.getTrackModel().trackSpots(id)
    for spot in track:
        sid = spot.ID()
        # Fetch spot features directly from spot.
        # Note that for spots the feature values are not stored in the FeatureModel
        # object, but in the Spot object directly. This is an exception; for tracks
        # and edges, you have to query the feature model.
        x=spot.getFeature('POSITION_X')
        y=spot.getFeature('POSITION_Y')
        t=spot.getFeature('FRAME')
        q=spot.getFeature('QUALITY')
        snr=spot.getFeature('SNR_CH1')
        mean=spot.getFeature('MEAN_INTENSITY_CH1')
        model.getLogger().log('\tspot ID = ' + str(sid) + ': x='+str(x)+', y='+str(y)+', t='+str(t)+', q='+str(q) + ', snr='+str(snr) + ', mean = ' + str(mean))

[java.lang.Enum.toString] 


TypeError: can only concatenate str (not "java.lang.String") to str

In [145]:
fm

<java object 'fiji.plugin.trackmate.FeatureModel'>